In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output, Video
import os

In [2]:
au = 1.496e11
G = 6.67e-11

In [3]:
class particle:
    def __init__(self, pos, velocity=0., mass=1.):
        self.pos = float(pos)
        self.velocity = float(velocity)
        self.mass = float(mass)
    
    def update_grav(self, accel):
        self.velocity += accel
        
    def move(self):
        self.pos += self.velocity

In [4]:
def get_colours(palette, n):
    clrs = []
    cmap = matplotlib.cm.get_cmap(palette, n)
    for i in range(cmap.N):
        rgb = cmap(i)[:3]
        clrs.append(matplotlib.colors.rgb2hex(rgb))
    return clrs

In [5]:
def grav(part_a, part_b):
    dist = part_b.pos - part_a.pos
    if dist != 0:
        on_a = G * part_b.mass / dist
    else:
        on_a = 0
    part_a.update_grav(on_a)
    part_b.update_grav(-on_a)

In [6]:
def step(parts):
    for aidx, aval in enumerate(parts):
        for bidx, bval in enumerate(parts[aidx:]):
            grav(aval, bval)
    for i in parts:
        i.move()

In [182]:
def plot_parts(parts, xlim=1*au, ylim=1*au, save=None, show=None, clrs='viridis'):
    fig = plt.figure(figsize=(10, 10))
    plt.xlim(-xlim, xlim)
    plt.ylim(-ylim, ylim)
    plt.scatter(*zip(*[(i.pos, i.velocity) for i in parts]), c=clrs)
    if show:
        plt.show()
    if save:
        plt.savefig(save)
    plt.close()

In [190]:
def integrate(parts, steps, savedir=None, start=0, xlim=1*au, ylim=1*au, clrs='viridis'):
    if start == 0:
        try: 
            !rm {savedir}/fig_*.png
        except:
            pass
    for i in np.arange(start, start+steps):
        if savedir:
            pth = f'{savedir}/fig_{i+1:04}.png'
        plot_parts(parts, xlim=xlim, ylim=ylim, save=pth, clrs=clrs)
        step(parts)
    !ffmpeg -y -framerate 30 -pattern_type glob -i '{savedir}/fig_*.png' -c:v libx264 -pix_fmt yuv420p {savedir}/out.mp4

In [287]:
def initialize(n_bodies, mass, spacing=1*au):
    init_pos = np.linspace(-spacing, spacing, n_bodies)
    init_parts = [particle(pos, mass=mass) for pos in init_pos]
    return init_parts

In [351]:
n_bodies = 200

clrs = get_colours('viridis', n_bodies)
parts1 = initialize(n_bodies=1, mass=1e35, spacing=0)
parts2 = initialize(n_bodies=n_bodies-1, mass=1e29, spacing=1*au)
parts = parts1 + parts2

In [352]:
integrate(parts, steps=50, savedir='grav', start=00, xlim=10*au, ylim=1*au, clrs=clrs)

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.16)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.2.2_1 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags='-I/Library/Java/JavaVirtualMachines/adoptopenjdk-13.0.1.jdk/Contents/Home/include -I/Library/Java/JavaVirtualMachines/adoptopenjdk-13.0.1.jdk/Contents/Home/include/darwin -fno-stack-check' --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libmp3lame --enable-libopus --enable-librubberband --enable-libsnappy --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librtmp --ena

In [353]:
Video(f'{workingdir}/out.mp4')